In [68]:
how = "dat"
import pickle
import numpy as np
from scipy.sparse import csr_matrix
with open("model/alldic.pickle", "rb") as f:
  alldic = pickle.load(f)
with open("model/allnum.pickle", "rb") as f:
  allnum = pickle.load(f)
with open("model/depmap.pickle", "rb") as f:
  depmap = pickle.load(f)
with open("Repository/Feature/gene2vec.pickle","rb") as f:
  gene2vec = pickle.load(f)
with open("model/datdic.pickle", "rb") as f:
  datdic = pickle.load(f)
with open("model/datnum.pickle", "rb") as f:
  datnum = pickle.load(f)
import gene_to_sym
gene_to_sym = gene_to_sym.get()

In [58]:
#sparse matrix
def make_spa(fdic,fnum,exptup, col,row, k, C):
  colap = col.append
  rowap = row.append
  for nt in exptup:
    fpre = fdic[nt[0].upper()]
    fnex = fdic[nt[1].upper()]
    count = 0
    for fp in fpre:
      ff = fnum[fp]
      if fp in fnex:
        colap(C+ff*3)
        count += 1
      else:
        colap(C+ff*3+1)
        count+=1
    for fn in fnex:
      ff = fnum[fn]
      if fn not in fpre:
        colap(C+ff*3+2)
        count+=1
    row += [k for _ in range(count)]
    k += 1
  C += len(fnum)*3
  return col,row,C

def makespa_wv(g2v,exptup, col,row, k, C):
  d = []
  colap = col.append
  rowap = row.append
  dap = d.append
  npc = np.corrcoef
  for nt in exptup:
    nt0,nt1 = nt[0].lower(),nt[1].lower()
    if (nt0 not in g2v) or (nt1 not in g2v):
       k += 1
       continue
    else:
       colap(C)
       dap(npc(g2v[nt0],g2v[nt1])[0][1])
       rowap(k)
       k += 1
  C += 1
  return col,row,C,d

def makespa_dep(depmap,exptup, col,row, k, C):
  d = []
  colap = col.append
  rowap = row.append
  dap = d.append
  npc = np.corrcoef
  for nt in exptup:
    nt0 = nt[0]
    nt1 = nt[1]
    if (nt0 not in depmap) or (nt1 not in depmap):
       k += 1
       continue
    else:
       colap(C)
       x = npc(depmap[nt0],depmap[nt1])[0][1]
       dap(x)
       rowap(k)
       k += 1
  C += 1
  return col,row,C,d


def expespar(exptup, ya, k):
    row = []
    col = []
    C = 0
    d = 0
    if how == "all":
       col,row,C = make_spa(alldic,allnum,exptup, col,row, k, C)
       col,row,C,d1 = makespa_wv(gene2vec,exptup, col,row, k, C)
       col,row,C,d2 = makespa_dep(depmap,exptup, col,row, k, C)
       data = [1 for _ in range(len(col)-len(d1)-len(d2))]+d1+d2
    elif how == "dat":
       col,row,C = make_spa(datdic,datnum,exptup, col,row, k, C)
       col,row,C,d2 = makespa_dep(depmap,exptup, col,row, k, C)
       data = [1 for _ in range(len(col)-len(d2))]+d2
    y = np.full(len(exptup),ya)
    k += len(exptup)
    return data, row, col, k, y, C

In [8]:
#generate tuples representing experimental contexts
from scipy.sparse import csr_matrix
import random
def make_tuple(path_to_csv,start,end,sampling=None,Nega=True):
  f = open(path_to_csv,"r")
  pmcid = []
  sentid = []
  genes = []

  for line in f:
    ls = line.strip().split(",")
    if ls[0].isnumeric():
      #Year Filter
      year = int(ls[0])
      if year > end or year < start:continue 

      #PMCID,sentid
      pmcid.append(ls[1])
      sentid.append(int(ls[2]))
      genes.append(ls[3])


  posi_tuple=[]
  nega_tuple=[]
  raw = len(pmcid)
  for n,p in enumerate(pmcid):
    for n2 in range(1000):
      nn = n+n2 
      if nn >= raw:break
      if p != pmcid[nn]: break #PMCID check
      if sentid[n]+1 < sentid[nn]: break #sent check

      if sentid[n] +1 == sentid[nn]: 
             g1,g2 = genes[n],genes[nn]
             posi_tuple.append((g1,g2))
  
             #Negative sampling
             if Nega:
                  negatives = random.sample(genes,7)
                  for neg in negatives:
                      if (g1,neg) not in posi_tuple_all:
                        nega_tuple.append((g1,neg))
                        
  if sampling != None:
    ret = random.sample(posi_tuple,sampling)
    if Nega:ret2 = random.sample(nega_tuple,sampling*5)
    return ret,ret2
  return posi_tuple,nega_tuple       


In [27]:
def make_csr(posi_tuple,nega_tuple):
  datap, row, col, kp, y, cp = expespar(posi_tuple, 1, 0)
  datan, rown, coln, kn, yn, cn = expespar(nega_tuple, 0, kp)
  row += rown
  col += coln
  data = datap+datan
  y =np.concatenate([y,yn])
  C = cn
  k = kn
  x = csr_matrix((data, (row, col)), (k, C))
  return x,y

In [64]:

def scoration(query,model):
    data = []
    row = []
    col = []
    pr = []
    sc = []
    extup = []
    
    for sym in set(gene_to_sym.values()):
        extup.append((query,sym))
        pr.append(sym)

    extups = []
    extups.append(extup[:10000]) #memory
    extups.append(extup[10000:20000])
    extups.append(extup[20000:30000])
    extups.append(extup[30000:40000])
    extups.append(extup[40000:])
        
    sco = []
    for n in range(len(extups)):
          data, row, col,k, y, C = expespar(extups[n],1,0)
          scot = model.predict(csr_matrix((data, (row,col)),(k, C))).tolist()
          sco += scot
          scot = []
        
    score = pd.DataFrame({"Gene":pr,"Score":sco})

    score = score.sort_values("Score",  ascending=False)
    score["Rank"] = list(range(1,len(score)+1))
    
    return score
    


In [60]:
import pandas as pd
path_to_csv = 'data/experiments_for_xgboost.csv'
path_to_dev = 'data/experiments_for_xgboost.csv'
trainys =2010
trainye=2021
froma = 3
fromp = 3
posi_tuple_all,n = make_tuple(path_to_csv,trainys,trainye,Nega=False)
posi_tuple_all = set(posi_tuple_all)
posi_tuple,nega_tuple = make_tuple(path_to_csv,trainys,trainye)
x,y=make_csr(posi_tuple,nega_tuple)

In [69]:
import xgboost as xgb
model =  xgb.XGBRegressor(objective="reg:logistic",eta=0.2,reg_lambda=0.1,max_depth=10)
model.fit(x,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.2, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.200000003, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=16, num_parallel_tree=1,
             objective='reg:logistic', random_state=0, reg_alpha=0,
             reg_lambda=0.1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [70]:
a=scoration("CEP57L1",model)
a[:30]

ValueError: Feature shape mismatch, expected: 253901, got 10000

In [ ]:
model.save_model('model/model.pickle')